In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from scipy import fft
import matplotlib.pyplot as plt

data = pd.read_csv('export_mobility.csv', sep=',')
data2 = pd.read_csv('export_mobility_v2.csv', sep=',')
data = data[['x', 'y', 'z', 'mode']]
data2 = data2[['x', 'y', 'z', 'mode']]
data = data.append(data2)
data = data.reset_index()
modes = list(data.groupby('mode').count().index[1:])
print(modes)

Using TensorFlow backend.


['2', '3', '5', 'BICYCLE', 'BUS', 'CAR', 'METRO', 'TRAIN', 'TRAM', 'WAIT', 'WALK']


In [2]:
i = 0
dict = {}
X_walk = None
X_nowalk = None
y_nowalk = None
y_walk = None
y = None
sum = 0
for mode in modes:
    if mode in ['BICYCLE', 'BUS', 'CAR', 'METRO', 'TRAM', 'WALK']:
        df_mode = data[data['mode'] == mode][['x', 'y', 'z']]
        n = df_mode.shape[0]
        sample_n = 20
        sum = int(sum + n/sample_n)
        print(sum)
        rest = n % sample_n
        df_mode.drop(df_mode.tail(rest).index, inplace=True)
        n = n-rest
        Xx = np.array(df_mode['x']).reshape(int(n / sample_n), sample_n)
        xfft = fft(np.array(df_mode['x']).reshape(int(n / sample_n), sample_n), axis=0)
        Xy = np.array(df_mode['y']).reshape(int(n / sample_n), sample_n)
        yfft = fft(np.array(df_mode['y']).reshape(int(n / sample_n), sample_n), axis=0)
        Xz = np.array(df_mode['z']).reshape(int(n / sample_n), sample_n)
        zfft = fft(np.array(df_mode['z']).reshape(int(n / sample_n), sample_n), axis=0)
        XXX = np.dstack((Xx, Xy, Xz, xfft, yfft, zfft))
        
        # assign X for both cases
        if X_walk is None:
            X_walk = XXX
        if X_nowalk is None and mode != "WALK":
            X_nowalk = XXX
        else:
            X_walk = np.append(X_walk, XXX, axis=0)
            if mode != "WALK":
                X_nowalk = np.append(X_nowalk, XXX, axis=0)

        # assign y for both cases as well as the objective one
        if y is None:
            y = np.full((int(n / sample_n)), i)
        else:
            y = np.append(y, np.full((int(n/sample_n)), i))
            
        if y_walk is None:
            if mode == "WALK":
                y_walk = np.full((int(n / sample_n)), 1)
            else:
                y_walk = np.full((int(n / sample_n)), 0)
        if y_nowalk is None and mode != "WALK":
            y_nowalk = np.full((int(n/sample_n)), i)
        else:
            if mode == "WALK":
                y_walk = np.append(y_walk,np.full((int(n/sample_n)), 1))
            else:
                y_walk = np.append(y_walk, np.full((int(n/sample_n)), 0))
                y_nowalk = np.append(y_nowalk, np.full((int(n/sample_n)), i))

        dict[mode] = i
        i = i+1
y_nowalk = to_categorical(y_nowalk)
y_walk = to_categorical(y_walk)
print(y.shape)

28
78
147
386
628
1663
(1663,)


In [3]:
model_walk = None
score_walk = None

kfold = KFold(5, True, 1)
# enumerate splits
for train, test in kfold.split(X_walk):
    
    x_train, x_test, y_train, y_test = X_walk[train], X_walk[test], y_walk[train], y_walk[test]
    
    verbose, epochs, batch_size = 0, 10, 32
    samples, features, outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(samples, features)))
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    # evaluate the model
    _, accuracy_test = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    _, accuracy_train = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=0)
    if score_walk is None:
        score_walk = accuracy_test
    elif accuracy_test > score_walk:
        score_walk = accuracy_test
        model_walk = model
    print("testing accuracy: ", accuracy_test, "training accuracy: ", accuracy_train)


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.966966986656189 training accuracy:  0.9969924688339233


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9519519805908203 training accuracy:  0.9789473414421082


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.966966986656189 training accuracy:  0.9902255535125732


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9879518151283264 training accuracy:  0.9939894676208496


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9789156913757324 training accuracy:  0.9909842014312744


In [6]:
model_walk.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 19, 64)            832       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 18, 64)            8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 18, 64)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 9, 64)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               57700     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

In [7]:
model_nowalk = None
score_nowalk = None

kfold = KFold(5, True, 1)
# enumerate splits
for train, test in kfold.split(X_nowalk):
    
    x_train, x_test, y_train, y_test = X_nowalk[train], X_nowalk[test], y_nowalk[train], y_nowalk[test]
    
    verbose, epochs, batch_size = 0, 10, 32
    samples, features, outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(samples, features)))
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    # evaluate the model
    _, accuracy_test = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    _, accuracy_train = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=0)
    if score_nowalk is None:
        score_nowalk = accuracy_test
    elif accuracy_test > score_nowalk:
        score_nowalk = accuracy_test
        model_nowalk = model
    print("testing accuracy: ", accuracy_test, "training accuracy: ", accuracy_train)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9158729910850525 training accuracy:  0.982868492603302


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.934920608997345 training accuracy:  0.9808765053749084


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9142857193946838 training accuracy:  0.9904382824897766


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9296000003814697 training accuracy:  0.9924452900886536


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


testing accuracy:  0.9424000382423401 training accuracy:  0.9888667464256287


## Table for the predictions as percentages

In [13]:
predictions = model_walk.predict(X_walk)
predictions_nonwalk = model_nowalk.predict(X_walk)

df = pd.DataFrame(predictions, columns=['WALK', 'OTHER'])
walk_idx = df.index[df['WALK'] > 0.5].tolist()
df_reclassify = pd.DataFrame(predictions_nonwalk, columns=[ 'BICYCLE', 'BUS', 'CAR', 'METRO', 'TRAIN', 'TRAM'])
df_reclassify.iloc[walk_idx] = 0

df_whole = pd.concat([df, df_reclassify], axis=1)
df = df_whole.drop(['OTHER'], axis=0)
df_whole

ValueError: Shape of passed values is (1663, 5), indices imply (1663, 6)

## Accuracy for Ensembled model

In [14]:
predictions_cl = model_walk.predict_classes(X_walk)
predictions_nonwalk_cl = model_nowalk.predict_classes(X_walk)
print(predictions_cl.shape, predictions_nonwalk_cl.shape, y.shape)

(1663,) (1663,) (1663,)


In [15]:
idx = np.where(predictions_cl == 1)
predictions_cl[idx] = 5
idx = np.where(predictions_cl == 0)
predictions_cl[idx] = predictions_nonwalk_cl[idx]
predictions_cl = np.array(predictions_cl)
y = np.array(y)
error = predictions_cl - y
erridx = np.where(error != 0)
error = len(erridx)/len(y)
print("error=", error, ", accuracy=", 1-error)

error= 0.0006013229104028864 , accuracy= 0.9993986770895971
